## Imports

In [ ]:
import os
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

## Load Network and Generate Sensors

In [ ]:
cubes = 'data_lak/cubes.lis'
sensors = generate_sensors(cubes)

network = 'data_lak/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [ ]:
W = np.eye(260)

## Calculate Initial Sigma0

In [ ]:
V = compute_residuals(cnet, sensors)
dof = J.shape[0] - J.shape[1]
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)

print(sigma0)

## Populate Jacobian

In [ ]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solvi
J, column_dict = compute_jacobian(cnet, sensors, parameters)

## Bundle Iteration

In [ ]:
N = J.T.dot(W).dot(J)
C = J.T.dot(W).dot(V)
dX = np.linalg.inv(N).dot(C)
print(dX)

## Redundancy Number

In [ ]:
# redundancy for every measure
# vector will hold same order as the measures in the cnet df
# def compute_measure_redundancy
Qxx = np.linalg.inv(N)
Qvv = np.linalg.inv(W) - J.dot(Qxx).dot(J.T)
r = np.diagonal(Qvv.dot(W))

print(f'Minimum redundancy: {min(r)}')
print(f'Maximum redundancy: {max(r)}')
plt.boxplot(r)

## Data Snooping

In [ ]:
# For data snooping we need to calculate updated residuals
qvv = np.diagonal(Qvv)
sigma_vi = sigma0*np.sqrt(qvv)
wi = -V/sigma_vi
# k = 3.29 #critical values from Forstner
k = 4.1 #cricital value from Baarda

# plt.figure()
# plt.boxplot(wi)
plt.figure()
plt.hist(wi[wi < k], bins=np.linspace(-4,5,50))
plt.hist(wi[wi > k], bins=np.linspace(-4,5,50))

## Update Sensors and Ground Points

In [ ]:
# update the sensor partials
for sn, sensor in sensors.items():
    n_param = len(parameters[sn])
    for i in range(n_param):
        sensor.setParameterValue(i, dX[column_dict[sn]+i])
        
# update ground points
for _, row in cnet.iterrows():
    point_id = row['id']
    ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
    adj = dX[column_dict[point_id]:column_dict[point_id]+3] 
    cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

## Whole bundle process in a loop

In [ ]:
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
W = np.eye(260)
tol = 1

##############

iteration = 0
V = compute_residuals(cnet, sensors)
dof = J.shape[0] - J.shape[1]
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

# while abs(sigma0-old_sigma0) < tol:
iterations = 3
for i in range(iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J, column_dict = compute_jacobian(cnet, sensors, parameters)    
    N = J.T.dot(W).dot(J) # calculate the normal equation
    C = J.T.dot(W).dot(V) 
    dX = np.linalg.inv(N).dot(C) #calculate change in camera parameters and ground points
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    # update the sensor parameters
    for sn, sensor in sensors.items():
        n_param = len(parameters[sn])
        for i in range(n_param):
            old_param = sensor.getParameterValue(i)
            sensor.setParameterValue(i, old_param+dX[column_dict[sn]+i])

    # update ground points
    for _, row in cnet.iterrows():
        point_id = row['id']
        ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
        adj = dX[column_dict[point_id]:column_dict[point_id]+3] 
        cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

    V = compute_residuals(cnet, sensors)
    dof = J.shape[0] - J.shape[1]
    sigma0 = np.sqrt((V.dot(W).dot(V))/dof)
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    